# Stock Market Trade Call Prediction

* Analysing stock data of HDFC bank and building a machine learning model for predicting the trade calls based on bollinger bands concept.

**First let's import all the necessary python modules**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


**Now let's read the data and perform some EDA steps**

In [2]:
infy_=pd.read_csv('HDFCBANK.NS (1).csv')

In [3]:
infy_

,Date,Open,High,Low,Close Price,Adj Close,Volume
0,15-05-2017,775.000000,778.599976,771.125000,776.700012,760.248596,1402390.0
1,16-05-2017,777.500000,781.000000,774.125000,779.825012,763.307373,1795188.0
2,17-05-2017,779.400024,782.700012,775.750000,778.575012,762.083923,1736236.0
3,18-05-2017,774.500000,780.750000,774.049988,778.549988,762.059326,1553640.0
4,19-05-2017,779.750000,784.950012,772.625000,780.625000,764.090393,1427068.0
...,...,...,...,...,...,...,...
800,12-08-2020,1065.000000,1066.000000,1048.000000,1063.699951,1063.699951,10333216.0
801,13-08-2020,1067.949951,1071.349976,1056.500000,1059.050049,1059.050049,9946370.0
802,14-08-2020,1065.900024,1065.900024,1027.300049,1034.449951,1034.449951,10462256.0
803,17-08-2020,1047.000000,1047.000000,1020.200012,1032.750000,1032.750000,9780506.0


In [4]:
infy_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         805 non-null    object 
 1   Open         804 non-null    float64
 2   High         804 non-null    float64
 3   Low          804 non-null    float64
 4   Close Price  804 non-null    float64
 5   Adj Close    804 non-null    float64
 6   Volume       804 non-null    float64
dtypes: float64(6), object(1)
memory usage: 44.1+ KB


In [5]:
infy_.describe()

,Open,High,Low,Close Price,Adj Close,Volume
count,804.000000,804.000000,804.000000,804.000000,804.000000,8.040000e+02
mean,1038.166380,1047.273008,1027.619934,1037.452916,1029.932716,7.146057e+06
std,130.022965,129.623812,129.189218,129.195180,132.842092,6.804195e+06
min,770.450012,778.599976,738.750000,767.700012,760.248596,2.850040e+05
25%,932.362503,939.987503,923.943756,931.549988,919.903473,3.031632e+06
50%,1034.649964,1044.640015,1020.489990,1032.625000,1027.739563,4.944403e+06
75%,1128.785004,1142.850006,1120.627472,1131.727478,1128.184570,8.344990e+06
max,1305.000000,1305.500000,1288.800049,1302.400024,1302.400024,5.406435e+07


In [6]:
infy_['index']=range(0,805)


In [7]:
infy_[infy_['Open'].isnull()]['index']


603    603
Name: index, dtype: int32

In [8]:
infy_.drop(infy_[infy_['index']== 603]['index'], inplace=True)

In [9]:
infy_.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 804
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         804 non-null    object 
 1   Open         804 non-null    float64
 2   High         804 non-null    float64
 3   Low          804 non-null    float64
 4   Close Price  804 non-null    float64
 5   Adj Close    804 non-null    float64
 6   Volume       804 non-null    float64
 7   index        804 non-null    int32  
dtypes: float64(6), int32(1), object(1)
memory usage: 53.4+ KB


In [10]:
infy_.describe().T

,count,mean,std,min,25%,50%,75%,max
Open,804.0,1.038166e+03,1.300230e+02,770.450012,9.323625e+02,1.034650e+03,1.128785e+03,1.305000e+03
High,804.0,1.047273e+03,1.296238e+02,778.599976,9.399875e+02,1.044640e+03,1.142850e+03,1.305500e+03
Low,804.0,1.027620e+03,1.291892e+02,738.750000,9.239438e+02,1.020490e+03,1.120627e+03,1.288800e+03
Close Price,804.0,1.037453e+03,1.291952e+02,767.700012,9.315500e+02,1.032625e+03,1.131727e+03,1.302400e+03
Adj Close,804.0,1.029933e+03,1.328421e+02,760.248596,9.199035e+02,1.027740e+03,1.128185e+03,1.302400e+03
Volume,804.0,7.146057e+06,6.804195e+06,285004.000000,3.031632e+06,4.944403e+06,8.344990e+06,5.406435e+07
index,804.0,4.017500e+02,2.325642e+02,0.000000,2.007500e+02,4.015000e+02,6.025000e+02,8.040000e+02


In [11]:
rolling_14=infy_['Close Price'].rolling(14).mean()

In [12]:
std_14=infy_['Close Price'].rolling(14).std()

In [13]:
infy_['Upper_band']=rolling_14+2*std_14

In [14]:
infy_['Lower_band']=rolling_14-2*std_14

In [15]:
infy_['rolling_14']=rolling_14

In [16]:
infy_.rolling_14.isnull().value_counts()

False    791
True      13
Name: rolling_14, dtype: int64

In [17]:
infy_.Upper_band.isnull().value_counts()

False    791
True      13
Name: Upper_band, dtype: int64

In [18]:
infy_.Lower_band.isnull().value_counts()

False    791
True      13
Name: Lower_band, dtype: int64

In [19]:
infy_.head().T

,0,1,2,3,4
Date,15-05-2017,16-05-2017,17-05-2017,18-05-2017,19-05-2017
Open,775,777.5,779.4,774.5,779.75
High,778.6,781,782.7,780.75,784.95
Low,771.125,774.125,775.75,774.05,772.625
Close Price,776.7,779.825,778.575,778.55,780.625
Adj Close,760.249,763.307,762.084,762.059,764.09
Volume,1.40239e+06,1.79519e+06,1.73624e+06,1.55364e+06,1.42707e+06
index,0,1,2,3,4
Upper_band,NaN,NaN,NaN,NaN,NaN
Lower_band,NaN,NaN,NaN,NaN,NaN


### Removing Null Values

As there are null values in features, we have to remove those null values.

In [20]:
infy_1=infy_.dropna(axis=0)


In [21]:
infy_1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 791 entries, 13 to 804
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         791 non-null    object 
 1   Open         791 non-null    float64
 2   High         791 non-null    float64
 3   Low          791 non-null    float64
 4   Close Price  791 non-null    float64
 5   Adj Close    791 non-null    float64
 6   Volume       791 non-null    float64
 7   index        791 non-null    int32  
 8   Upper_band   791 non-null    float64
 9   Lower_band   791 non-null    float64
 10  rolling_14   791 non-null    float64
dtypes: float64(9), int32(1), object(1)
memory usage: 71.1+ KB


In [22]:
infy_1['index']=range(0,791)

C:\Users\nallu\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
infy_1


,Date,Open,High,Low,Close Price,Adj Close,Volume,index,Upper_band,Lower_band,rolling_14
13,01-06-2017,819.500000,819.924988,810.275024,814.299988,797.052124,1679770.0,0,829.698951,760.965339,795.332145
14,02-06-2017,817.000000,823.049988,813.099976,817.125000,799.817261,1606772.0,1,832.635858,763.803429,798.219644
15,05-06-2017,816.849976,819.474976,812.500000,817.775024,800.453552,1449950.0,2,835.082621,766.778097,800.930359
16,06-06-2017,817.500000,821.900024,815.000000,819.474976,802.117493,2501316.0,3,836.740316,770.963254,803.851785
17,07-06-2017,820.200012,822.450012,815.174988,820.400024,803.023010,1277138.0,4,837.344247,776.337899,806.841073
...,...,...,...,...,...,...,...,...,...,...,...
800,12-08-2020,1065.000000,1066.000000,1048.000000,1063.699951,1063.699951,10333216.0,786,1112.152694,997.854455,1055.003575
801,13-08-2020,1067.949951,1071.349976,1056.500000,1059.050049,1059.050049,9946370.0,787,1094.619876,1006.808713,1050.714294
802,14-08-2020,1065.900024,1065.900024,1027.300049,1034.449951,1034.449951,10462256.0,788,1088.842897,1006.149970,1047.496434
803,17-08-2020,1047.000000,1047.000000,1020.200012,1032.750000,1032.750000,9780506.0,789,1078.872782,1008.420082,1043.646432


In [24]:
infy_1['Close Price'].value_counts()

1050.650024    3
1061.319946    2
1028.800049    2
923.125000     2
1264.750000    2
              ..
1244.349976    1
901.174988     1
1288.800049    1
1047.469971    1
849.000000     1
Name: Close Price, Length: 770, dtype: int64

## Building features from other attributes

As we are using the bollinger bands concept for trade call prediction, we need to calculate bollinger bands, they are only our features for classifying the trade calls.

In [25]:
def Call_fun_test(ind):
    cp=test_1[test_1['index']==ind]['Close']
    lp=test_1[test_1['index']==ind]['Lower_band']
    mid=test_1[test_1['index']==ind]['rolling_14']
    up=test_1[test_1['index']==ind]['Upper_band']
    if (cp<lp).bool():
        return 'Buy'
    elif (lp<cp).bool() and (cp<mid).bool():
        return 'Hold'
    elif (cp>up).bool():
        return 'Short'   
    if (mid<cp).bool() and (cp<up).bool():
        return 'Hold'

In [26]:
def Call_fun(ind):
    cp=infy_1[infy_1['index']==ind]['Close Price']
    lp=infy_1[infy_1['index']==ind]['Lower_band']
    mid=infy_1[infy_1['index']==ind]['rolling_14']
    up=infy_1[infy_1['index']==ind]['Upper_band']
    if (cp<lp).bool():
        return 'Buy'
    elif (lp<cp).bool() and (cp<mid).bool():
        return 'Hold'
    elif (cp>up).bool():
        return 'Short'   
    if (mid<cp).bool() and (cp<up).bool():
        return 'Hold'
        

In [27]:
call=infy_1['index'].apply(Call_fun)

In [28]:
infy_1['Call']=call

C:\Users\nallu\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
infy_1.head().T


,13,14,15,16,17
Date,01-06-2017,02-06-2017,05-06-2017,06-06-2017,07-06-2017
Open,819.5,817,816.85,817.5,820.2
High,819.925,823.05,819.475,821.9,822.45
Low,810.275,813.1,812.5,815,815.175
Close Price,814.3,817.125,817.775,819.475,820.4
Adj Close,797.052,799.817,800.454,802.117,803.023
Volume,1.67977e+06,1.60677e+06,1.44995e+06,2.50132e+06,1.27714e+06
index,0,1,2,3,4
Upper_band,829.699,832.636,835.083,836.74,837.344
Lower_band,760.965,763.803,766.778,770.963,776.338


In [30]:
def change_call(n):
    if n =='Buy':
        return 0
    elif n=='Hold':
        return 1
    elif n=='Short':
        return 2

In [31]:
def change_call_t(n):
    if n =='Buy':
        return 0
    elif n=='Hold':
        return 1
    elif n=='Short':
        return 2

In [32]:
infy_1['Calls']=infy_1['Call'].apply(change_call)


C:\Users\nallu\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
infy_1['Calls'].value_counts()


1    724
2     43
0     24
Name: Calls, dtype: int64

### Extracting features and label

In [34]:
x=infy_1[['Upper_band','rolling_14',"Lower_band",'Close Price']]


In [35]:
y=infy_1['Calls']

In [36]:
import sklearn
sklearn.__version__

'0.22.1'

In [37]:
x.shape,y.shape

((791, 4), (791,))

## Splitting data into train and test sets

Here we are splitting data as 70% train data and 30% test data

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.3,random_state=67)

## Feature scaling

* To avoid the dominance of features having higher numeric ranges over features with small numeric ranges, we scale all the features to the same range(mostly 0-1)
* We are using StandardScaler class for scaling the features

In [40]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [41]:
scaler=StandardScaler()

In [42]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [43]:
X_scaled=scaler.transform(X_train)

In [44]:
X_scaled.mean(axis=0)

array([-1.08572986e-15, -6.93839199e-16,  6.42443703e-16, -1.07288098e-15])

In [45]:
X_scaled.std(axis=0)

array([1., 1., 1., 1.])

In [46]:
X_test=preprocessing.scale(X_test)

## Developing the Model

I have used the support vector classifier(SVC) to classify the trade calls.

In [47]:
from sklearn.svm import SVC

In [48]:
smod=SVC(probability=True)

In [49]:
smod.fit(X_scaled,Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [50]:
y_score = smod.fit(X_scaled, Y_train).decision_function(X_test)


## Evaluating the model

In [51]:
from sklearn.metrics import confusion_matrix

In [52]:
smod.score(X_test,Y_test)

0.9117647058823529

In [53]:
preds=smod.predict(X_test)
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [54]:
confusion_matrix(Y_test,preds)

array([[  0,   8,   0],
       [  0, 217,   0],
       [  0,  13,   0]], dtype=int64)

In [55]:
smod.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': True,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

Our model is performing very poorly, so we need to improve the model.

In [56]:
from sklearn.metrics import classification_report
print(classification_report(preds,Y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.91      0.95       238
           2       0.00      0.00      0.00         0

    accuracy                           0.91       238
   macro avg       0.33      0.30      0.32       238
weighted avg       1.00      0.91      0.95       238



C:\Users\nallu\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Improving the SVC Model

 Here, I am using the GridSearchCV to improve the model

In [57]:
from sklearn.model_selection import GridSearchCV

In [58]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]} 

In [59]:
grid = GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=2)

In [60]:
grid.fit(X_scaled,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ...........

[CV] ................................... C=100, gamma=1, total=   0.0s
[CV] C=100, gamma=0.1 ................................................
[CV] ................................. C=100, gamma=0.1, total=   0.0s
[CV] C=100, gamma=0.1 ................................................
[CV] ................................. C=100, gamma=0.1, total=   0.0s
[CV] C=100, gamma=0.1 ................................................
[CV] ................................. C=100, gamma=0.1, total=   0.0s
[CV] C=100, gamma=0.1 ................................................
[CV] ................................. C=100, gamma=0.1, total=   0.0s
[CV] C=100, gamma=0.1 ................................................
[CV] ................................. C=100, gamma=0.1, total=   0.0s
[CV] C=100, gamma=0.01 ...............................................
[CV] ................................ C=100, gamma=0.01, total=   0.0s
[CV] C=100, gamma=0.01 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.2s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [61]:
grid.score(X_test,Y_test)

0.9285714285714286

In [62]:
preds_g=grid.predict(X_test)

In [63]:
print(classification_report(Y_test,preds_g))

              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.93      1.00      0.96       217
           2       1.00      0.08      0.14        13

    accuracy                           0.93       238
   macro avg       0.91      0.52      0.57       238
weighted avg       0.93      0.93      0.91       238



Here, we can observe that our model got improved, and is now performing well.